In [ ]:
import cv2
import numpy as np
np.random.seed(42)
from my_function import imshow

## 噪声

### 噪声分类

- 高斯噪声
  - 高斯噪声的特点是增加或减少原始信号，使得原始信号出现了随机“抖动”，并且抖动范围服从高斯分布（正态分布）
- 椒盐噪声
  - 椒盐噪声又称脉冲噪声，它随机改变一些像素值，在二值图像上表现为使一些像素点变白，一些像素点变黑。 是由图像传感器，传输信道，解码处理等产生的黑白相间的亮暗点噪声

### 噪声生成

对于高斯噪声：

- 使用`np.random.normal`生成指定形状的正态分布数组
- 将原图与噪声相加
- 使用`np.clip`将数组中的元素限制在指定的最小值和最大值之间，超出范围的值会被替换为边界值，确保像素值在0~255
- 增强噪声：增大正态分布标准差

In [6]:
# 读取灰度图
img=cv2.imread("./images/lena_gray.png",cv2.IMREAD_GRAYSCALE) # 以灰度模式读取

def generate_gaussian_noise(img,std=25):
    gauss_noise=np.random.normal(0,std,size=img.shape)
    img_G_noise=np.clip(img+gauss_noise,0,255)
    return img_G_noise

img_G_noise=generate_gaussian_noise(img,std=25)
cv2.imwrite("./images/lena_guass.png",img_G_noise)

True

对于椒盐噪声：

- 使用`np.random.rand`生成[0, 1)区间的均匀分布的随机数组
- 随机数组各值与图像各像素值对应，随机数大于某个值对应像素值置0，小于某个值对应像素值置255
- 增强噪声：改变阈值

In [8]:
def generate_salt_and_pepper_noise(img,threshold=0.05):
    x,y=img.shape
    salt_pepper_noise=np.random.rand(x,y)
    img[salt_pepper_noise<threshold]=0    # 5%椒噪声
    img[salt_pepper_noise>(1-threshold)]=255  # 5%盐噪声
    img_S_noise=img.copy()
    return img_S_noise

img_S_noise=generate_salt_and_pepper_noise(img,threshold=0.05)
cv2.imwrite("./images/lena_salt_pepper.png", img_S_noise)

True

## 图像平滑处理

在图像处理中，平滑处理（也称为模糊处理）是一种常见的操作，用于减少图像中的噪声或细节

opencv提供以下几种平滑处理技术：

- 均值滤波 blur()
- 高斯滤波 GaussianBlur()
- 中值滤波 medianBlur()
- 双边滤波 bilateralFilter()

滤波器对图像进行卷积运算

### 均值滤波 blur()
  - 将图像中每个像素的值替换为其周围像素的平均值
  - 均值滤波可以有效地去除噪声，但可能会导致图像变得模糊

cv2.blur(image, (5, 5)):
- image: 输入的图像。
- (5, 5): 滤波核的大小，表示在水平和垂直方向上取平均值的范围。

In [ ]:
noise=cv2.imread("./images/lena_guass.png")

blur=cv2.blur(noise,(5,5))
imshow(noise=noise,blur=blur)

### 高斯滤波 GaussianBlur()

- 高斯滤波是一种基于高斯函数的平滑处理方法
- 与均值滤波不同，高斯滤波在计算像素平均值时，会给中心像素赋予更高的权重，而给边缘像素赋予较低的权重
- 高斯滤波适用于去除图像中的高斯噪声，能够更好地保留图像的边缘信息

cv2.GaussianBlur(image, (5, 5), 0):
- image: 输入的图像
- (5, 5): 滤波核的大小
- 0: 高斯核的标准差，如果为0，则根据核大小自动计算

In [37]:
noise=cv2.imread("./images/lena_guass.png")

Gauss=cv2.GaussianBlur(noise,(5,5),0)
imshow(blur=blur,Gauss=Gauss)

### 中值滤波 medianBlur()

- 中值滤波是一种非线性平滑处理方法,它的原理是将图像中每个像素的值替换为其周围像素的中值
- 中值滤波在去除椒盐噪声（即图像中随机出现的黑白点）时非常有效，并且在保留图像边缘信息方面表现较好

cv2.medianBlur(image, 5):
- image: 输入的图像。
- 5: 滤波核的大小，必须是奇数

In [36]:
noise=cv2.imread("./images/lena_salt_pepper.png")

median=cv2.medianBlur(noise,5)
imshow(noise=noise,median=median)

### 双边滤波 bilateralFilter()

- 双边滤波是一种非线性的平滑处理方法，它结合了空间邻近度和像素值相似度
- 与高斯滤波不同，双边滤波在平滑图像的同时，能够保留图像的边缘信息。这是因为双边滤波不仅考虑像素之间的空间距离，还考虑像素值之间的差异

cv2.bilateralFilter(image, 9, 75, 75)：
- image: 输入的图像
- 9: 滤波核的大小
- 75: 颜色空间的标准差，控制像素值相似度的权重
- 75: 坐标空间的标准差，控制空间距离的权重

In [38]:
noise=cv2.imread("./images/lena_guass.png")

bilateral=cv2.bilateralFilter(noise,9,75,75)
imshow(Gauss=Gauss,bilateral=bilateral)

### 总结

- 均值滤波简单易用，但可能导致图像模糊
- 高斯滤波在去除噪声的同时能较好地保留边缘信息
- 中值滤波特别适合去除椒盐噪声
- 双边滤波则在保留边缘信息方面表现出色